In [6]:
import pandas as pd

df = pd.read_csv('train.csv')

In [7]:
# count how many missing values in each column
print(df.isnull().sum())

#count how many double entries in each column
print("Double entries: ", df.duplicated().sum())

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64
Double entries:  0
